In [4]:
from utils import HParam
hp = HParam('config/finetune_baichuan.yaml')
hp.llama.train_batchsize = hp.data.train_batchsize
from llama_model import LlamaPeftGenerate, LlamaModule
model,tokenizer = LlamaPeftGenerate(hp.llama,hp.lora)


Using pad_token, but it is not set yet.


trainable params: 4194304 || all params: 7004753920 || trainable%: 0.05987796356449307


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("baichuan-inc/Baichuan-7B",cache_dir = hp.llama.cache_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("baichuan-inc/Baichuan-7B", device_map="cpu", trust_remote_code=True,cache_dir = hp.llama.cache_dir)



14时23分;山地地形;敌方坦克出现在侧面战场;我方直升机进入维修状态;
5.6秒后,敌人开始攻击。此时,在地图上显示的“战斗”区域内有两个目标:一个是在右边的小型建筑物中被击中的小型车辆(红色),另一个是位于左边的重型装甲车(绿色)。由于该地区没有其他障碍


In [9]:
inputs = tokenizer('14时23分;山地地形;敌方坦克出现在侧面战场;我方直升机进入维修状态;', return_tensors='pt')
#inputs = inputs.to('cuda:0')
pred = model.generate(**inputs, max_new_tokens=256,repetition_penalty=1.1)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))

14时23分;山地地形;敌方坦克出现在侧面战场;我方直升机进入维修状态;
5.6秒后,敌人开始攻击。此时,在地图上显示的“战斗”区域内有两个目标:一个是在右边的小型建筑物中被击中的小型车辆(红色),另一个是位于左边的重型装甲车(绿色)。由于该地区没有其他障碍物或掩体,因此可以很容易地看到它们之间的距离和位置关系。此外,还可以通过观察周围环境来判断其移动方向以及可能采取的行动方式等信息。


In [10]:
gpt2_litmodel = LlamaModule
print('load from checkpoint')
gpt2_litmodel = gpt2_litmodel.load_from_checkpoint('/root/autodl-tmp/ckpt/20230719_ft_baichuan/epoch=epoch=0020-validation.loss=val_loss=4.911.ckpt', 
                                                   args=hp.llama, model=model,tokenizer=tokenizer)

load from checkpoint


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 gpt2_litmodel = LlamaModule                                                                  │
│   2 print('load from checkpoint')                                                                │
│ ❱ 3 gpt2_litmodel = gpt2_litmodel.load_from_checkpoint('/root/autodl-tmp/ckpt/20230719_ft_ba     │
│   4 │   │   │   │   │   │   │   │   │   │   │   │      args=hp.llama, model=model,tokenizer=     │
│   5                                                                                              │
│                                                                                                  │
│ /root/miniconda3/lib/python3.8/site-packages/pytorch_lightning/core/module.py:1531 in            │
│ load_from_checkpoint                                                                             │
│                                                                                                  │
│   1528 │   │   │   pretrained_model.freeze()                                                     │
│   1529 │   │   │   y_hat = pretrained_model(x)                                                   │
│   1530 │   │   """                                                                               │
│ ❱ 1531 │   │   loaded = _load_from_checkpoint(                                                   │
│   1532 │   │   │   cls,                                                                          │
│   1533 │   │   │   checkpoint_path,                                                              │
│   1534 │   │   │   map_location,                                                                 │
│                                                                                                  │
│ /root/miniconda3/lib/python3.8/site-packages/pytorch_lightning/core/saving.py:60 in              │
│ _load_from_checkpoint                                                                            │
│                                                                                                  │
│    57 │   **kwargs: Any,                                                                         │
│    58 ) -> Union["pl.LightningModule", "pl.LightningDataModule"]:                                │
│    59 │   with pl_legacy_patch():                                                                │
│ ❱  60 │   │   checkpoint = pl_load(checkpoint_path, map_location=map_location)                   │
│    61 │                                                                                          │
│    62 │   # convert legacy checkpoints to the new format                                         │
│    63 │   checkpoint = _pl_migrate_checkpoint(                                                   │
│                                                                                                  │
│ /root/miniconda3/lib/python3.8/site-packages/lightning_fabric/utilities/cloud_io.py:51 in _load  │
│                                                                                                  │
│   48 │   │   )                                                                                   │
│   49 │   fs = get_filesystem(path_or_url)                                                        │
│   50 │   with fs.open(path_or_url, "rb") as f:                                                   │
│ ❱ 51 │   │   return torch.load(f, map_location=map_location)  # type: ignore[arg-type]           │
│   52                                                                                             │
│   53                                                                                             │
│   54 def get_filesystem(path: _PATH, **kwargs: Any) -> Abst

In [ ]:
pred = gpt2_litmodel.model.generate(**inputs, max_new_tokens=256,repetition_penalty=1.1)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))